In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
logs_df_statistical_method = pd.read_csv("/content/drive/MyDrive/logs_df_statistical_method.csv")
logs_df_statistical_method.head()

,Unnamed: 0,request_processing_time,target_processing_time,response_processing_time,received_bytes,sent_bytes,number of requests
0,0,0.000,0.759,0.0,375,542,1
1,1,0.001,0.194,0.0,374,2363,4
2,2,0.001,0.444,0.0,374,542,7
3,3,0.000,0.221,0.0,375,5641,7
4,4,0.001,0.168,0.0,375,2532,8


In [ ]:
logs_df_statistical_method.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
logs_df_statistical_method.head()

,request_processing_time,target_processing_time,response_processing_time,received_bytes,sent_bytes,number of requests
0,0.000,0.759,0.0,375,542,1
1,0.001,0.194,0.0,374,2363,4
2,0.001,0.444,0.0,374,542,7
3,0.000,0.221,0.0,375,5641,7
4,0.001,0.168,0.0,375,2532,8


In [ ]:
logs_df_statistical_method.shape

(2577066, 6)

In [3]:
test_dict = dict()
test_dict.setdefault("test") == None

True

In [ ]:
# statistical model unidimensional case
anomalies_features = list()
anomalies_indices = list()
indices_leverage_statistic = dict() # for multidimensional case
for column in logs_df_statistical_method.columns:
    for i in tqdm(range(1, logs_df_statistical_method.shape[0])):
        if indices_leverage_statistic.setdefault() == None:
          indices_leverage_statistic.setdefault(i, list())
        if i == 1:
          current_mean = np.mean(logs_df_statistical_method[column][:2])
          current_var = np.var(logs_df_statistical_method[column][:2])
        else:
          current_mean = ((i - 1) * previous_mean + logs_df_statistical_method[column][i]) * 1/i
          current_var = ((i - 1) * previous_var + np.power(logs_df_statistical_method[column][i] - current_mean, 2)) * 1/i
        leverage_statistic = 1/i + np.power(logs_df_statistical_method[column][i] - current_mean, 2) / current_var
        indices_leverage_statistic[i].append(leverage_statistic) # for multidimensional case
        if leverage_statistic > (2 * 2) / i: # hi > 2 * (p + 1) / n
            anomalies_features.append(column)
            anomalies_indices.append(i)
        previous_mean = current_mean
        previous_var = current_var

  0%|          | 0/2577065 [00:00<?, ?it/s]<ipython-input-7-8fc8ef7ab71a>:12: RuntimeWarning: invalid value encountered in double_scalars
  if (1/i + np.power(logs_df_statistical_method[column][i] - current_mean, 2) / current_var) > (2 * 2) / i: # hi > 2 * (p + 1) / n
100%|██████████| 2577065/2577065 [01:20<00:00, 32092.24it/s]


In [ ]:
logs_df_statistical_method.shape[0]

2577066

In [ ]:
# analyzing the output
np.unique(np.array(anomalies_features), return_counts=True)

(array(['number of requests', 'received_bytes', 'request_processing_time',
        'response_processing_time', 'sent_bytes', 'target_processing_time'],
       dtype='<U24'),
 array([2577062, 2576889, 2575527, 2575694, 2571239, 2559206]))

In [ ]:
''' # statistical model multidimensional case
anomalies_indices_md = list()
columns = logs_df_statistical_method.columns
for i in range(1, logs_df_statistical_method.shape[0]):
    print(i)
    leverage_statistics_ith_observation = list()
    for column in columns:
      if i == 1:
        current_mean = np.mean(logs_df_statistical_method[column][:2])
        current_mean = np.mean(logs_df_statistical_method[column][:2])
      else:
        current_mean = ((i - 1) * previous_mean + logs_df_statistical_method[column][i]) * 1/i
        current_var = ((i - 1) * previous_var + np.power(logs_df_statistical_method[column][i] - current_mean, 2)) * 1/i
      leverage_statistics_ith_observation.append((1/i + np.power(logs_df_statistical_method[column][i] - current_mean, 2)) / current_var)
      previous_mean = current_mean
      previous_var = current_var
    if np.mean(leverage_statistics_ith_observation) > (2 * len(columns)) / i: # mean(hi) > 2 * (p + 1) / n
      anomalies_indices_md.append(i) '''

In [ ]:
# statistical model multidimensional case
anomalies_indices_md = list()
num_of_columns = logs_df_statistical_method.shape[1]
for i in tqdm(range(1, logs_df_statistical_method.shape[0])):
  if np.mean(indices_leverage_statistic[i]) > (2 * len(num_of_columns)) / i: # mean(hi) > 2 * (p + 1) / n
    anomalies_indices_md.append(i)

In [ ]:
# analyzing the output
len(anomalies_indices_md)

In [ ]:
set(anomalies_indices).intersection(set(anomalies_indices_md))

In [ ]:
set(anomalies_indices).difference(set(anomalies_indices_md))